- Nice example from
    - [youtrack](https://youtrack.jetbrains.com/issue/KTNB-261/Create-example-notebook-showcasing-running-JVM-tests-inside-it-and-obtaining-its-results)

- Todo
    - analyze more (USE, onClassAnnotation, SessionOptions, etc)

In [ ]:
SessionOptions.resolveMpp = true

In [ ]:
USE {
    dependencies {
        implementation("org.junit.jupiter:junit-jupiter-api:5.8.0")
        implementation("org.junit.jupiter:junit-jupiter-engine:5.8.0")
        implementation("org.junit.platform:junit-platform-launcher:1.9.3")
        implementation("io.kotest:kotest-assertions-core:5.6.2")
    }
}

In [ ]:
import org.junit.platform.engine.TestExecutionResult
import org.junit.platform.launcher.TestExecutionListener
import org.junit.platform.launcher.TestIdentifier
import org.junit.platform.launcher.TestPlan

class ConsoleTestExecutionListener : TestExecutionListener {
    override fun testPlanExecutionStarted(testPlan: TestPlan) {
        // Initialization code if needed
    }

    override fun executionStarted(testIdentifier: TestIdentifier) {
        // Called when a test execution starts
        println("Running test: ${testIdentifier.displayName}")
    }

    override fun executionFinished(
        testIdentifier: TestIdentifier,
        testExecutionResult: TestExecutionResult
    ) {
        // Called when a test execution finishes
        println("Test [${testIdentifier.displayName}]: ${testExecutionResult.status}")
        val throwable = testExecutionResult.throwable.orElse(null)
        if (throwable != null) {
            println("Failure reason: ${throwable.stackTraceToString()}")
        }
    }

    override fun testPlanExecutionFinished(testPlan: TestPlan?) {
        // Cleanup code if needed
    }
}

In [ ]:
import org.junit.jupiter.api.Test
import org.junit.platform.launcher.core.LauncherFactory
import org.junit.platform.engine.discovery.DiscoverySelectors
import org.junit.platform.launcher.LauncherDiscoveryRequest
import org.junit.platform.launcher.core.LauncherDiscoveryRequestBuilder
import kotlin.reflect.KClass

fun runTestClass(kClass: KClass<*>) {
    val launcher = LauncherFactory.create()
    launcher.registerTestExecutionListeners(ConsoleTestExecutionListener())
    val request: LauncherDiscoveryRequest = LauncherDiscoveryRequestBuilder.request()
        .selectors(DiscoverySelectors.selectClass(kClass.java))
        .build()

    val testPlan = launcher.discover(request)
    
    launcher.execute(testPlan)
}

In [ ]:
annotation class MyTest
USE {
    onClassAnnotation<MyTest> { classes ->
        classes.forEach { runTestClass(it) }
    }
}

In [ ]:
blabla 

In [ ]:
import io.kotest.matchers.shouldBe

@MyTest
class MyTestClass1 {
    // Define your test methods here
    @Test
    fun test1() {
        println(123)
        (6 * 7) shouldBe 43
    }
}